## Import required packages

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import bs4
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import time
from selenium.common.exceptions import NoSuchElementException

# Develop a simple version first

## Get the link & Basic setting

In [2]:
link = 'https://hk.jobsdb.com/hk/search-jobs/data/1'

#Setting up the driver
driver = webdriver.Chrome('/path/to/chromedriver')
driver.get(link)

C:\Users\Yan-PC\AppData\Local\Temp\ipykernel_8648\3270838411.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/path/to/chromedriver')


In [3]:
# Page 1 first and second job Xpath Comparsion  
#//*[@id="jobList"]/div[2]/div[2]/div/div[1]/div/div/article/div/div/div[1]
#//*[@id="jobList"]/div[2]/div[2]/div/div[2]/div/div/article/div/div/div[1]

# Page 2 first job Xpath
#//*[@id="jobList"]/div[2]/div[2]/div/div[1]/div/div/article/div/div/div[1]

#Find the job Xpath
Clicks = driver.find_element(By.XPATH, '//*[@id="jobList"]/div[2]/div[2]/div/div[1]/div/div/article/div/div/div[1]')

#Click into the job
Clicks.click()



### Read the required data from the web

In [103]:
# Setting for bs4
time.sleep(3)
driver.switch_to.window(driver.window_handles[0])
get_url = driver.current_url
jobsdb = requests.get(get_url).text
soup = BeautifulSoup(jobsdb, 'html.parser')

In [2]:
# Create empty dataframe
df_job = pd.DataFrame(columns=['Job_Title', 'Job_Highlight', 'Job_Description', 'Job_Requirement'])

# Creating the dictionary for storing the data
dic_job = { 'Job_Title' : [], 
           'Job_Highlight' : [], 
           'Job_Description' : [],
           'Job_Requirement' : []
            
}

In [105]:
# Place store the job title 
dic_job['Job_Title'] = driver.find_element(By.TAG_NAME, 'h1').text

# Place store the job highlight 
highlight_web = soup.find('ul', 'sx2jih0 sx2jih3 _17fduda0 _17fduda6').find_all('span','sx2jih0 zcydq84u es8sxo0 es8sxo1 es8sxo21 _1d0g9qk4 es8sxoa')
highlight_lst= [word.text for word in highlight_web]
dic_job['Job_Highlight'] = highlight_lst

# Jr and Jd are in the same section in the html code Thus, I find them all tgt
# Find the location of the jr and jd
jr_web = soup.find('div', '_1x1c7ng0').find('div', 'sx2jih0').find_all('ul')
lst_li = [word.find_all('li') for i, word in enumerate (jr_web) if i != 3]
# Locate jr and jd in which span respectively
lst_span_jr = [word.find_all('span') for word in lst_li[0]]
lst_span_jd = [word.find_all('span') for word in lst_li[1]]
# Get the text into the list
lst_jr = [word[0].text for word in lst_span_jr ]
lst_jd = [word[0].text for word in lst_span_jd ]
# Add jr and jd into dictionary
dic_job['Job_Requirement'] = lst_jr
dic_job['Job_Description'] = lst_jd

# Add the new observation to the dataframe
df_job = df_job.append(dic_job, ignore_index=True)

print(df_job)


                                          Job_Title  \
0  ASSISTANT MANAGER – ANALYTICS & DATA ENGINEERING   

                                       Job_Highlight  \
0  [ETL & automate data process, Data analytics o...   

                                     Job_Description  \
0  [Organize and prepare regular business analysi...   

                                     Job_Requirement  
0  [Bachelor’s degree in Computer Science, Inform...  


C:\Users\Yan-PC\AppData\Local\Temp\ipykernel_26200\248005108.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_job = df_job.append(dic_job, ignore_index=True)


# Try a more complex version

In [22]:
# Create empty dataframe
df_job = pd.DataFrame(columns=['Job_Title', 'Location','Job_Highlight', 'JRJD'])

# Creating the dictionary for storing the data
dic_job = { 'Job_Title' : [],
           'Company_Name' : [],
           'Location' : [], 
           'Job_Highlight' : [], 
           'JRJD' : []
}

In [23]:
# define a function for checking if the jr and jd are contain in <span> or <p>
class find_jrjd:

    def __init__(self, soup):
        self.soup = soup
    
    # check if there are ul. If no, find <p> to deal with it
    def check_ul(self):
        try:
            find_ul = self.soup.find('div', '_1x1c7ng0').find_all('ul')
            return 'with ul' 
        except NoSuchElementExceptionError:
            return 'no ul'
    
    def get_no_ul(self):
        try:
            find_span = self.soup.find('div', '_1x1c7ng0').find_all('span')
            span_ind = 1
        except NoSuchElementExceptionError:
            find_p = self.soup.find('div', '_1x1c7ng0').find_all('p')
            p_ind = 1
        if span_ind == 1:
            content = [ word.text for i, word in enumerate(find_span) if i < 3]
        elif p_ind == 1:
            content = [ word.text for i, word in enumerate(find_p)]
        return content
    
    def get_word_in_ul(self):
        find_ul = self.soup.find('div', '_1x1c7ng0').find_all('ul')
        li_text = [word.text for word in find_ul[:2] ]
        return li_text
    
    def with_loc(self):
        try:
            location = self.soup.find('div', 'z1s6m00 _1hbhsw66i').find('div','z1s6m00 _1hbhsw65a').find('span', 'z1s6m00 _1hbhsw64y y44q7i0 y44q7i1 y44q7i21 y44q7ii')
            #location = self.soup.find('div', 'z1s6m00 _1hbhsw65a').find('span')
            return location.text
        except AttributeError:
            return 'Without_location'
        except NoSuchElementException:
            return 'Without_location'

    def get_name(self):
        try:
            find_name = self.soup.find('div', 'z1s6m00 _5135ge0 _5135ge6').find('span', 'z1s6m00 _1hbhsw64y y44q7i0 y44q7i2 y44q7i21 _1d0g9qk4 y44q7ia')
            return find_name.text
        except AttributeError:
            return 'Unknown'

def click(job_path):
    try:
        button = driver.find_element(By.XPATH, job_path)
        return 'valid'
    except NoSuchElementException:
        return 'invalid'
        

In [31]:
# Input the page no. to conduct the crawling
page = int(input('Please input the page numer!'))

for i in range(page):
    # Give the link
    link = f'https://hk.jobsdb.com/hk/search-jobs/data/{i+33}'

    #Setting up the driver
    driver = webdriver.Chrome('/path/to/chromedriver')
    driver.get(link)
    time.sleep(1.5)

    for var_job_no in range(30):
        #Find the job Xpath
        job_path = f'//*[@id="jobList"]/div/div[2]/div/div[{var_job_no+1}]/div/div/article/div/div/div[1]/div[1]/div[2]/h1/a'
        if click(job_path) == 'valid':
            #Action for clicking
            click_action = Keys.CONTROL + Keys.ENTER
            #For clicking button
            button = driver.find_element(By.XPATH, job_path)
            button.send_keys(click_action)
            time.sleep(2)
        else:
            continue

        if len(driver.window_handles) == 2:

            driver.switch_to.window(driver.window_handles[1])
            get_url = driver.current_url
            jobsdb = requests.get(get_url).text
            soup = BeautifulSoup(jobsdb, 'html.parser')

            # Place store the job title 
            dic_job['Job_Title'] = driver.find_element(By.TAG_NAME, 'h1').text

            # Company Name
            dic_job['Company_Name'] = soup.find('div', 'z1s6m00 _5135ge0 _5135ge6').find('span', 'z1s6m00 _1hbhsw64y y44q7i0 y44q7i2 y44q7i21 _1d0g9qk4 y44q7ia').text

            # Place store the job highlight 
            highlight_web = soup.find('ul', 'z1s6m00 z1s6m03 _5135ge0 _5135ge6').find_all('li')
            highlight_lst= [word.text for word in highlight_web]
            dic_job['Job_Highlight'] = highlight_lst

            # Get the JD JR
            var = find_jrjd(soup)

            # Get the job location
            dic_job['Location'] = var.with_loc()
            
            if var.check_ul == 'with ul':
                text_jrjd = var.get_word_in_ul()
            else:
                text_jrjd = var.get_no_ul()
            dic_job['JRJD'] = text_jrjd

            # Add the new observation to the dataframe
            df_job = df_job.append(dic_job, ignore_index=True)

            # Back to the menu page 
            driver.close()
            time.sleep(1)
            
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(1)

        if click(job_path) == 'invalid':
            # Back to the menu page                   
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(1)

 

C:\Users\Yan-PC\AppData\Local\Temp\ipykernel_11060\259471174.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/path/to/chromedriver')
C:\Users\Yan-PC\AppData\Local\Temp\ipykernel_11060\259471174.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_job = df_job.append(dic_job, ignore_index=True)
C:\Users\Yan-PC\AppData\Local\Temp\ipykernel_11060\259471174.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_job = df_job.append(dic_job, ignore_index=True)
C:\Users\Yan-PC\AppData\Local\Temp\ipykernel_11060\259471174.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_job = df_job.append(dic_job, ignore_index=True)
C:\Users\Yan-PC\AppData\Local

In [32]:
df_job

,Job_Title,Location,Job_Highlight,JRJD,Company_Name
0,ASSISTANT MANAGER - DATA ANALYTICS,Wan Chai,"[Min 5-7 years’ experience in data analytics, ...",[RequirementsBachelor degree in Computer Scien...,Sun Hung Kai Properties
1,IT Assistant (Data Management),Mong Kok,"[5-days work, bank holiday, Data administratio...",[Job Responsibilities:To perform regular updat...,EIDEALINK LIMITED
2,"Associate Vice President, Dashboard Reporting ...",Wan Chai,"[Degree in Statistics, Business Analysis or re...",[ResponsibilitiesDirect report to Chief Analyt...,United Asia Finance Limited
3,Data Center Customer Operations Engineer / Fie...,Tsuen Wan Area,"[Open to Fresh Graduate applicants, Operation ...",[Who are we?Equinix is one of the fastest grow...,Equinix Hong Kong Ltd
4,IT Data Centre Operator,Tsuen Wan Area,"[Monitor & manage data center facilities, Fres...",[ What you’ll be doing?Perform activities for ...,OCBC Wing Hang Bank Limited
...,...,...,...,...,...
1216,"Business Intelligence Manager - Call Center , ...",Central,"[Call center project experience, 60K, provide ...",[Principal responsibilitiesFormulate effective...,NewPages Recruitment Company Limited
1217,Date Architect,Central,[3+years exp in trade platform develop& mainte...,[Responsibilities：Design and implement system ...,Manpower Services (Hong Kong) Limited
1218,"Assistant Vice President, Business Analytics &...",Without_location,"[Drive and maintain data governance framework,...",[Responsibilities:Drive and maintain data gove...,Chong Hing Bank Ltd
1219,Assurance-Forensic&Integrity Services-E-discov...,Without_location,"[Bachelor's degree in a numerate discipline, 8...",[Job summaryOur innovative Forensic Technology...,EY


In [57]:
df_job.to_csv('./Data/DataJob.csv', index=False, encoding='utf_8_sig')